![Astrofisica Computacional](../logo.png)

---
## 02. SQL query at the SDSS server


Eduard Larrañaga (ealarranaga@unal.edu.co)

---


### About this notebook

In this worksheet we will present some basic examples using SQL at the SDSS Server. 

---

### SQL

SQL (**S**tructured **Q**uery **L**anguage) is a standard language for storing, manipulating and retrieving data in databases.
 
 It is particularly useful in handling structured data, i.e. data incorporating relations among entities and variables.

### SDSS

The **S**loan **D**igital **S**ky **S**urvey (SDSS) has created the most detailed three-dimensional maps of the Universe ever made, with deep multi-color images of one third of the sky, and spectra for more than three million astronomical objects. 

https://www.sdss.org

Data Release 17 (DR17) is the final data release of the fourth phase of the Sloan Digital Sky Survey (SDSS-IV). DR17 contains SDSS observations through January 2021.

Data Release 16 includes six types of data: 
- images
- optical spectra (SDSS/SEGUE/BOSS/SEQUELS/eBOSS)
- infrared spectra (APOGEE/APOGEE-2)
- IFU spectra (MaNGA)
- stellar library spectra (MaStar)
- catalog data (parameters measured from images and spectra, such as magnitudes and redshifts).
 

### Accessing the Database of the DR17

Wde can access the data in the DR17 using SQL at the webpage

http://skyserver.sdss.org/dr17/SearchTools/sql



The information on all sky objects (stars, galaxies, etc) is stored in a database. A request to a database to return some information is called a **query**. When you ask a database for information, you must write your query in some computer language that the database can understand, for example SQL. 

When you write a query with SQL, the database searches for all records that meet your search criteria; in the case of SDSS, each record is the information on a single observation of a single object in the sky.

IMPORTANT: Since the database contains hundreds of pieces of information for each of millions of objects, a poorly written query could return Gigabytes of information that won't help you at all!. 

The best possible query will return all the information that you need, and no information that you don't need. *Writing queries is an art - before you start writing, you need to think carefully about what information you need.*

### The Database Structure

The SkyServer database is divided into a series of tables. Each table contains data of a different type. For example, all data related to an object's spectrum are stored in a table called "specObj." All data related to an object's image properties are stored in a table called "photoObj". 

SkyServer contains many tables, although most of the commonly accessed data are stored in just three or four of them.

Each table contains a series of data columns. A column contains only one type of data - for example, magnitude or sky position. Each record consists of a series of columns, although some columns may be empty for some records. Some tables contain only two or three columns; some contain hundreds! Knowing exactly which table and which column contains the data you want can be a challenge.

The complete schema of the database is described at

http://skyserver.sdss.org/dr17/MoreTools/browser

To browse through the tables, click on "Tables" in the right-hand column. 



The Schema Browser contains complete documentation on every table in the SkyServer database. But, most of the queries you will write will search through only a small subset of those tables. 

Some of the most commonly used tables (or views) are:

- **PhotoObj** - stores information about the images of every object, including run, rerun, camcol, field, ra, dec, magnitudes and object flags.

- **PlateX** - stores information on the aluminum plates that the SDSS uses to take spectra, including their exposure times and reddening information. You will need to find the Plate and MJD in this table to look up an object's spectrum in the Get Spectra tool.

- **SpecObj** - stores information on objects' spectra, including redshifts and spectroscopic classifications.



**Note:** SkyServer contains several subsets of the PhotoObj table. PhotoPrimary contains only the "best" measurements of each object. Generally, it's better to use PhotoPrimary rather than PhotoObj, which contains both good and bad data. Star contains only data for stars, Galaxy contains only data for galaxies, and Unknown contains only data for objects classified as "unknown." 

---
### A SQL Query

We will make a simple query at the SDSS server

http://skyserver.sdss.org/dr17/SearchTools/sql

An SQL query consists of three pieces or blocks: the **SELECT block**, the **FROM block** and the **WHERE block**.

 - The SELECT block tells the database which columns of data you want it to return. You must separate each column name with a comma. For example, if you wanted to find the celestial coordinates right ascension (ra) and declination (dec) of an object, the select block might read
```
SELECT ra, dec
```

- The FROM block specifies which table (or tables) you want to search. If you wanted to retrieve information from the specObj table, the FROM block would read
```
FROM
specObj
```

- The WHERE block allows you to search for records with certain characteristics. Your list of characteristics must be separated by boolean operators. Suppose you wanted to limit your search to a patch of sky with ra between 140 and 141 degrees and dec between 20 and 21 degrees. To search only this patch of sky, your where block would read
```
WHERE
ra BETWEEN 140 AND 141 AND
dec BETWEEN 20 AND 21
```

The complete query described above is
```
SELECT 
  ra, dec
FROM
  specObj
WHERE
  ra BETWEEN 140 AND 141 AND
  dec BETWEEN 20 AND 21
```

and will return only those records that have an ra between 140 and 141 and a dec between 20 and 21. When we do not restrict the number of results, the server will return only 500000 samples! However, for this query the results are much less than this limit.

The results can be received in various formats, including

- HTML
- XML
- CSV
- JSON
- VOTable
- FITS
- MyDB (This option stores the information in the server)

If you want the query to return fewer results, you can add the statement "TOP N" to the select block, where n is some integer from 1 to 1,000.
```
SELECT TOP 25
  ra, dec
FROM
  specObj
WHERE
  ra BETWEEN 140 AND 141 AND
  dec BETWEEN 20 AND 21
```


**Exercise**

1. Modify the previous query so it will return ra and dec of objects where the ra is between 140.25 and 140.75 and the dec is between 20.25 and 20.75. How many objects did the query return? Does this seem to make sense, given the smaller area you searched here? 

Open the Schema Browser and click on views. Select the table SpecObj. You will see a list of all the features (columns) in the SpecObj table, along with short descriptions of what they mean and if applicable, what values the column may have.
For instance, if you scroll down to the "class" column, its description says that it may have one of the following values: 'STAR', 'GALAXY', or 'QSO'. 

2. Modify your query so that it returns the ra, dec, and best object ID for galaxies (and only galaxies) whose ra is between 140.25 and 140.75 and whose dec is between 20.25 and 20.75.

---
### Other Sample Queries

```
SELECT TOP 10
  z, ra, dec, bestObjID
FROM
  specObj
WHERE
  class = 'galaxy' 
  AND z > 0.3 
  AND zWarning = 0
  ```

This query will look in the specObj table (which contains spectroscopic information) and return the first 10 object's features redshift (z), the right ascension, the declination, and the (final, or "best") object ID. The conditions imposed are to look only at galaxies with redshifts above 0.3 and no known problems with the redshift determination (zWarning = 0).

```
SELECT TOP 10 
    z, ra, dec, bestObjID
FROM
    specObj
WHERE
    (class = 'galaxy' OR class = 'qso')
    AND z > 0.3 
    AND zWarning = 0
```

This modification will add the quasars objects to the results.

```
SELECT TOP 10
  ra, dec, modelMag_u, modelMag_g, modelMag_r, modelMag_i, modelMag_z, objID
FROM
  photoObj
WHERE
  type = 6 
  AND modelMag_u - modelMag_g < 0.5
```

This query returns the first 10 photometric objects (in the PhotoObj table) that are labeled as stars (denoted by type = 6) with the color index  u - g < 0.5 and a limit  in the magnitudes such that modelMag_u - modelMag_g < 0.5.

**Exercises**

1. **Reddest galaxies in the area of sky near ra = 141.** Write a query to search for galaxies between ra = 140.9 and ra = 141.1 brighter than g = 18.0 for which the (u-g) color index satisfies u - g > 2.2. Retrieve the Object ID, ra, dec, and the five final magnitudes.

2. **Highest-redshift quasars in the SDSS database.** Write a query to search for quasars for which we have obtained spectra (specObj table) with redshifts greater than 4.5 and good measurements (zWarning = 0). Retrieve each quasar's Photo ID, ra, dec, and redshift. Which one is the quasar with the greater value of z?